# Recording Studio

This notebook is intended to record, preprocess and save the audios that will be later used by pyramidman assistant. It will make use of the speech recognizing as well for practical purposes, but theses will not be explained in this notebook, but rather in the third one.

This notebook focuses on making a proper listener in another thread that writes the audio data into a Queue that is later consumed by the main thread and in having a simple Recording Studio for making the audios for pyramidman

In [1]:
%load_ext autoreload
%autoreload 2

from pyramidman.audio_parameters import AudioParameters
from pyramidman.basic_audio_IO import play_audio, record_audio
from pyramidman.audio_utils import get_available_microphones, get_sysdefault_microphone_index, get_all_devices_str
from pyramidman.queue_utils import record_with_queue
from pyramidman.unwrapper import unwrap
from pyramidman.speech_recognizing import recognize_speech_from_mic
from pyramidman.hieroglyph import plot_timeseries_range_slider, create_tabs, plot_spectrogram
from pyramidman.hieroglyph import add_word_annotations

from pyramidman.Ihy import get_audio_menu_wav_file
from pyramidman.signal_processing import get_spectrogram

from pyramidman.queue_utils import put_audio_data_in_queue_callback_closure, listen_in_a_thread
from pyramidman.audio_utils import calibrate_microphone

import speech_recognition as sr
from pyramidman.deepspeech_tools import transcribe, DeepSpeechArgs

import plotly
import time

%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt

import plotly.graph_objs as go
from IPython.display import display

import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
import ipywidgets as widgets

from queue import Queue
import noisereduce as nr
import librosa

/home/montoya/anaconda3/envs/python36/lib/python3.7/site-packages/tqdm/autonotebook.py:17: TqdmExperimentalWarning:

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)



# Instantiate and calibrate microphone

Ideally, we would like a background process in a thread that whenever a sentence is finished, it is translated and plotted. This is the following code.

In [41]:
audio_params = AudioParameters()
audio_params.set_sysdefault_microphone_index()
audio_params.set_default_input_parameters()

In [42]:
audio_params.input_device_index = 6
audio_params.sample_rate = 48000

In [43]:
mic = audio_params.get_microphone()
r = sr.Recognizer()

calibrate_microphone(mic, r, duration = 3, dynamic_energy_threshold= False )

Calibrating microphone for 3 seconds.
Calibrated


In [51]:
unwrap(mic, max_level = 1)

<Microphone>	object has children:
    <module>	pyaudio_module
    <int>	device_index:	6
    <int>	format:	8
    <int>	SAMPLE_WIDTH:	2
    <int>	SAMPLE_RATE:	48000
    <int>	CHUNK:	1024
    <PyAudio>	audio
    <NoneType>	stream:	None

  <PyAudio>	audio has children:

  <module>	pyaudio_module has children:




### Minimize noise expetiments

In [5]:
file_to_record = "../audios/temp/recording.wav"
record_audio(audio_params, seconds = 5, filename = file_to_record)

Recording
Finished recording


In [6]:
play_audio(audio_params, file_to_record)

In [7]:

# load data
rate, data = wavfile.read(file_to_record)
data = data.astype(float)
# select section of data that is noise
initial_noise_duration = 1
noisy_part = data[:int(audio_params.sample_rate/initial_noise_duration)]

# perform noise reduction
reduced_noise = nr.reduce_noise(audio_clip=data, noise_clip=noisy_part, verbose=False)

In [8]:
reduced_noise = reduced_noise.astype(np.int16)

In [9]:
file_to_record_reduced = "../audios/temp/recording_reduced.wav"
wavfile.write(file_to_record_reduced, audio_params.sample_rate, reduced_noise)

In [10]:
tabs = get_audio_menu_wav_file(file_to_record)
display(tabs)

    'data': [{'line': {'color': 'deepskyblue'},
              'name': 'AAPL High'…

In [11]:
tabs = get_audio_menu_wav_file(file_to_record_reduced)
display(tabs)

    'data': [{'line': {'color': 'deepskyblue'},
              'name': 'AAPL High'…

In [12]:
# Load some audio
# Trim the beginning and ending silence

yt, index = librosa.effects.trim(reduced_noise.astype(float),top_db=20, ref=np.max, frame_length=512*4, hop_length=256*4)
print(yt.shape, reduced_noise.shape)
index

(239616,) (239616,)


array([     0, 239616])

In [13]:
file_to_record_reduced_cut = "../audios/temp/recording_reduced_cut.wav"
wavfile.write(file_to_record_reduced_cut, audio_params.sample_rate, yt.astype(np.int16))

In [14]:
tabs = get_audio_menu_wav_file(file_to_record_reduced_cut)
display(tabs)

    'data': [{'line': {'color': 'deepskyblue'},
              'name': 'AAPL High'…

## Transcribe to know which approach is better

In [15]:
play_audio(audio_params, file_to_record)
play_audio(audio_params, file_to_record_reduced)
play_audio(audio_params, file_to_record_reduced_cut)

In [16]:
args = DeepSpeechArgs()

In [17]:
transcribe(args, file_to_record)["sentence"]

'i'

In [18]:
transcribe(args, file_to_record_reduced)["sentence"]

'i'

In [19]:
transcribe(args, file_to_record_reduced_cut)["sentence"]

'i'

# Listen in background

Create a thread that records in the background and puts the sentences read into queue that has as input the 

# Tuning the parameters of the listening.

In [36]:
r.non_speaking_duration = 0.2
r.pause_threshold = 1.2
r.energy_threshold = 2500

### Initializing data

In [39]:
# Stop listening will stop the thread
q = Queue()
put_audio_data_in_queue_callback = put_audio_data_in_queue_callback_closure(r, mic, q)
stop_listening = listen_in_a_thread(r, mic, put_audio_data_in_queue_callback, phrase_time_limit = 10)

Exception in thread Thread-11:
Traceback (most recent call last):
  File "/home/montoya/anaconda3/envs/python36/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/home/montoya/anaconda3/envs/python36/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/montoya/Desktop/VScode/pyramidman/pyramidman/queue_utils.py", line 83, in threaded_listen
    with source as s:
  File "/home/montoya/anaconda3/envs/python36/lib/python3.7/site-packages/speech_recognition/__init__.py", line 134, in __enter__
    assert self.stream is None, "This audio source is already inside a context manager"
AssertionError: This audio source is already inside a context manager



### Consume the audios put in the queue

In [40]:
i = 0
while(True):
    
    filename_mic = f'../audios/temp/{i}.wav'
    audio = q.get()
    
    print("Saving... ", end = "")
    with open(filename_mic, "wb") as f:
        f.write(audio.get_wav_data())

    """     
    if i == 0:
        tabs = get_audio_menu_wav_file(filename_mic)
        display(tabs)
    else:
        tabs_next = get_audio_menu_wav_file(filename_mic)
        tabs.children = tabs_next.children
    """
    
    print("Transcribing...: ", end="")
    args = DeepSpeechArgs()
    metadata = transcribe(args, filename_mic)

    print(metadata["sentence"])
    # add_word_annotations(tabs.children[0],metadata["words"])
    
    if "pyramid man" in metadata["sentence"]:
        print("Activated broh")
        play_audio(audio_params, filename_mic)
    
    i+=1 

KeyboardInterrupt: 

### Stop listening

In [38]:
# calling this function requests that the background listener stop listening
stop_listening(wait_for_stop=False)
q.empty()

True

## Recording studio.

We have created a simple plotly UI to record and save the audios for the pyramidman assistant. This can be reused in the future for extension of capabilities.

In [ ]:
mic = audio_params.get_microphone()
r = sr.Recognizer()

figure_box = widgets.Box([go.FigureWidget()])

def record_button_callback(button):
    if button.description == "Start":
        filename_mic = '../audios/temp/hello_world.wav'

        with mic as source:
            # audio = r.record(source,duration = 2)
            audio = r.listen(source)
            
        with open(filename_mic, "wb") as f:
            f.write(audio.get_wav_data())
        
        figure_box.children = [get_audio_menu_wav_file(filename_mic)]
        
button_record = widgets.Button(
    value=False,
    description='Start',
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    icon='check'
)

button_record.on_click(record_button_callback)

                      
recorder_box = widgets.VBox([button_record, figure_box])
display(recorder_box)